In [1]:
from pathlib import Path
import numpy as np
import torch
from typing import List
from torch.nn.utils.rnn import pad_sequence
from mltrainer import rnn_models, Trainer
from torch import optim

from mads_datasets import datatools
import mltrainer
mltrainer.__version__

'0.2.5'

In [2]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor
preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

2025-10-04 22:48:15.934 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\mwien\.cache\mads_datasets\gestures
100%|██████████| 651/651 [00:00<00:00, 3248.32it/s]


In [3]:
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))

In [4]:
from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

accuracy = Accuracy()


In [5]:
loss_fn = torch.nn.CrossEntropyLoss()

In [6]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

# on my mac, at least for the BaseRNN model, mps does not speed up training
# probably because the overhead of copying the data to the GPU is too high
# so i override the device to cpu
device = "cpu"
# however, it might speed up training for larger models, with more parameters

using cpu


Set up the settings for the trainer and the different types of logging you want

In [7]:
settings = TrainerSettings(
    epochs=16, # increase this to about 100 for training
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs = {
        "save": True, # save every best model, and restore the best one
        "verbose": True,
        "patience": 5, # number of epochs with no improvement after which training will be stopped
        "delta": 0.0, # minimum change to be considered an improvement
    }
)
settings

epochs: 16
metrics: [Accuracy]
logdir: gestures
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>, <ReportTypes.MLFLOW: 'MLFLOW'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': True, 'verbose': True, 'patience': 5, 'delta': 0.0}

In [8]:
import torch.nn as nn
import torch
from torch import Tensor
from dataclasses import dataclass

@dataclass
class ModelConfig:
    input_size: int
    hidden_size: int
    num_layers: int
    output_size: int
    dropout: float = 0.0

class GRUmodel(nn.Module):
    def __init__(
        self,
        config,
    ) -> None:
        super().__init__()
        self.config = config
        self.rnn = nn.GRU(
            input_size=config.input_size,
            hidden_size=config.hidden_size,
            dropout=config.dropout,
            batch_first=True,
            num_layers=config.num_layers,
        )
        self.linear = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x: Tensor) -> Tensor:
        x, _ = self.rnn(x)
        last_step = x[:, -1, :]
        yhat = self.linear(last_step)
        return yhat

In [9]:
import mlflow
from datetime import datetime

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("gestures")
modeldir = Path("gestures").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)
    
n_repeats = 3

for repeat in range(n_repeats):
    print(f"Repeat {repeat+1}/{n_repeats}")
    
    with mlflow.start_run():
        # Set MLflow tags to record metadata about the model and developer
        mlflow.set_tag("model", f"{repeat}_GRU_16epochs_0.5drop_256hidsize")
        mlflow.set_tag("dev", "Marcello")
        # Log hyperparameters to MLflow

        mlflow.log_param("epochs", settings.epochs)

        mlflow.log_param("learning_rate", settings.optimizer_kwargs.get("lr", None))
        
        config = ModelConfig(
            input_size=3,
            hidden_size=256,
            num_layers=1,
            output_size=20,
            dropout=0.5,
        )

        model = GRUmodel(
            config=config,
        )

        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optim.Adam,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau,
            device=device,
        )
        trainer.loop()

        if not settings.earlystop_kwargs["save"]:
            tag = datetime.now().strftime("%Y%m%d-%H%M-")
            modelpath = modeldir / (tag + "model.pt")
            torch.save(model, modelpath)

2025/10/04 22:48:18 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/04 22:48:18 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
c:\Master Applied Data Science\Year 2\Semester 3 (Deep Learning & Model Deployment)\Portfolio-Marcello-Wienhoven\.venv\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(


Repeat 1/3


2025-10-04 22:48:18.387 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures\20251004-224818
2025-10-04 22:48:19.543 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:01<00:00, 44.66it/s]
2025-10-04 22:48:21.658 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.5166 test 2.2426 metric ['0.2219']
2025-10-04 22:48:21.658 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (2.2426 --> 2.2426).Saving gestures\20251004-224818\checkpoint.pt ...
100%|██████████| 81/81 [00:02<00:00, 36.41it/s]
2025-10-04 22:48:24.140 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.9809 test 1.7016 metric ['0.3969']
2025-10-04 22:48:24.140 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (2.2426 --> 1.7016).Saving gestures\20251004-224818\checkpoint.pt ...
100%|██████████| 81/81 [00:02<00:00, 31.89it/s]
2025-10-04 22:48:26.929 | INFO

Repeat 2/3


100%|██████████| 81/81 [00:01<00:00, 43.62it/s]
2025-10-04 22:48:56.455 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.5499 test 2.3305 metric ['0.1953']
2025-10-04 22:48:56.456 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (2.3305 --> 2.3305).Saving gestures\20251004-224854\checkpoint.pt ...
100%|██████████| 81/81 [00:01<00:00, 43.57it/s]
2025-10-04 22:48:58.503 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.9897 test 1.7936 metric ['0.3703']
2025-10-04 22:48:58.505 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (2.3305 --> 1.7936).Saving gestures\20251004-224854\checkpoint.pt ...
100%|██████████| 81/81 [00:02<00:00, 40.14it/s]
2025-10-04 22:49:00.733 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.3335 test 1.0822 metric ['0.5859']
2025-10-04 22:49:00.734 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (1.7936 --> 1.0822).Saving gestures\20251004-224854\checkpoint.pt ...
100%|█████

Repeat 3/3


100%|██████████| 81/81 [00:02<00:00, 31.10it/s]
2025-10-04 22:49:35.833 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.4903 test 2.1510 metric ['0.2094']
2025-10-04 22:49:35.835 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (2.1510 --> 2.1510).Saving gestures\20251004-224932\checkpoint.pt ...
100%|██████████| 81/81 [00:02<00:00, 32.35it/s]
2025-10-04 22:49:38.588 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.9125 test 1.8049 metric ['0.3641']
2025-10-04 22:49:38.588 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (2.1510 --> 1.8049).Saving gestures\20251004-224932\checkpoint.pt ...
100%|██████████| 81/81 [00:02<00:00, 32.05it/s]
2025-10-04 22:49:41.360 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.3847 test 1.1612 metric ['0.5891']
2025-10-04 22:49:41.360 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (1.8049 --> 1.1612).Saving gestures\20251004-224932\checkpoint.pt ...
100%|█████

In [10]:
mlflow.end_run()

## Open MLFlow
"""
```bash
mlflow server \
    --backend-store-uri sqlite:///mlflow.db \
    --host 127.0.0.1 \ 
    --port 5000 \
        
mlflow server --backend-store-uri sqlite:///3-hypertuning-rnn/mlflow.db --host 127.0.0.1 --port 5000
```
"""